In [11]:
import re

# 3rd part imports
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from dashboard.logic.io import GSHEETS_URL, read_gsheet, comment_button, total_assets, \
    total_value_to_num

# Plotting

In [29]:
import pandas as pd
import numpy as np

# bokeh
from bokeh.io import show, output_notebook
from bokeh.palettes import Category10
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Legend, LabelSet, LegendItem, HoverTool
from bokeh.layouts import column

In [11]:
output_notebook()

Loading BokehJS ...

In [2]:
from dashboard.route.investments import df_plot

In [6]:
df_plot

,Asset_Class,Total_Value
0,Gold/Silver (physical),217695.30
1,Gold/Silver (allocated),1211234.47
2,Business Equity,420335.60
3,Real Estate Equity (IOM/UK),1146588.80
4,Real Estate Equity (US),482000.00
5,Stocks,0.00
6,"Tangibles (Antiques, Art, etc)",35000.00
7,Cash,50000.00
8,NaN,NaN


In [49]:
def pie_chart(
    df: pd.DataFrame, 
    x: str, 
    y: str,
    x_hover: str=None,
    y_hover: str=None, 
    radius: float=0.8,
    x_range: tuple[float, float]=(-1, 1.0),
    percentage_decimal: int=1,
    label_distance: float=3,
    fig_height: int=350,
    background_color: str='#212529',
    pallette: dict=Category10,
    sizing_mode='scale_both',
    hover_tooltip: str='default',
    legend_place: str='right',
    fig_kwargs: dict={},
    wedge_kwargs: dict=dict(line_width=1.5, alpha=0.7),
    legend_kwargs: dict=dict(location='center', click_policy="hide",
                             label_text_color='white', border_line_width=0,
                             inactive_fill_color='#9fcf2e', inactive_fill_alpha=0.15),
    label_kwargs: dict=dict(text_font_size='10pt', text_align='center')
    ):
    
    
    # sort df by "y"
    df = df.sort_values(by=y, ignore_index=True)
    
    # calculate sector start and end angles
    df['angle'] = df[y] / df[y].sum() * 2 * np.pi
    df['cumsum_start'] = df['angle'].cumsum(axis='rows').shift(1).fillna(0)
    df['cumsum_end'] = df['angle'].cumsum(axis='rows')
    
    # calculate y percentages for hover & labels
    df['percentage_number'] = (df[y] / df[y].sum() * 100).round(percentage_decimal)
    df['percentage_hover'] = df['percentage_number'].astype(str)
    df['percentage_label'] = df['percentage_number'].apply(lambda x: "" if x < 5 else f"{x:.{percentage_decimal}f}%")
    
    # project label text coordinates to polar coordinates
    df['label_x_pos'] = np.cos(df['angle'].cumsum() - df['angle'].div(2)) * label_distance * radius/4
    df['label_y_pos'] = np.sin(df['angle'].cumsum() - df['angle'].div(2)) * label_distance * radius/4
    
    # remove assets that are 0
    df = df[df[y] > 0]
    
    # reset dataframe index to start with 0
    df = df.reset_index(drop=True)
    
    print(df)
    
    # init the figure/canvas for the plot
    p = figure(height=fig_height, 
               toolbar_location=None, 
               x_range=x_range,
               y_range=(-1.0, 1.0),
               sizing_mode=sizing_mode,
               **fig_kwargs)
    
    legend_items = []
    for idx, color in enumerate(pallette[df.shape[0]]):
        
        source = ColumnDataSource(df.iloc[idx,:].to_frame().T)
        
        # create the glyphs renderers
        wedge = p.wedge(x=0, y=0, radius=radius, start_angle="cumsum_start", 
                        end_angle="cumsum_end", source=source, **wedge_kwargs,
                        fill_color=color, hover_fill_color=color,
                        line_color=background_color, hover_line_color=background_color,
                        line_alpha=1, hover_alpha=1, hover_line_alpha=1)
        
        label = LabelSet(x='label_x_pos', y='label_y_pos', text='percentage_label',
                         source=source, level='glyph', text_color=background_color, **label_kwargs)
        
        x_hover = x if x_hover is None else x_hover
        y_hover = y if y_hover is None else y_hover
        
        hover_tooltip = hover_tooltip if hover_tooltip != 'default' else \
            f"""
                <div>
                    <p style="margin:0;font-weight:bold;color:grey;">@{x_hover}</p>
                    <p style="padding:0;margin:0;font-weight:bold;">$@{y_hover}{{0,0.00}} (@percentage_hover%)</p>
                </div>
            """
        
        p.add_layout(label)
        p.add_tools(HoverTool(renderers=[wedge],
                              tooltips=hover_tooltip))

        legend_items.append(LegendItem(label=df[x][idx], renderers=[wedge]))

    # legend
    legend = Legend(items=legend_items, **legend_kwargs,
                    background_fill_color=background_color) 
    p.add_layout(legend, place=legend_place)

    
    # figure attributes
    p.toolbar.active_drag = None
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    
    
    p.min_border=0
    p.outline_line_alpha=0
    p.outline_line_width=0
    p.outline_line_color = p.background_fill_color = p.border_fill_color = background_color

    return p

show(
    pie_chart(
        df=df_plot,
        x='Asset_Class',
        y='Total_Value',
        sizing_mode='fixed',
        legend_place='below',
        radius=0.5,
        fig_height=600
        
    )
)

                      Asset_Class  Total_Value     angle  cumsum_start  \
0  Tangibles (Antiques, Art, etc)     35000.00  0.061723      0.000000   
1                            Cash     50000.00  0.088176      0.061723   
2          Gold/Silver (physical)    217695.30  0.383911      0.149900   
3                 Business Equity    420335.60  0.741273      0.533811   
4         Real Estate Equity (US)    482000.00  0.850019      1.275084   
5     Real Estate Equity (IOM/UK)   1146588.80  2.022039      2.125103   
6         Gold/Silver (allocated)   1211234.47  2.136043      4.147142   

   cumsum_end  percentage_number percentage_hover percentage_label  \
0    0.061723                1.0              1.0                    
1    0.149900                1.4              1.4                    
2    0.533811                6.1              6.1             6.1%   
3    1.275084               11.8             11.8            11.8%   
4    2.125103               13.5             13.5        

# Stocks Watchlist

In [7]:
STOCKS_WATCH_URL = "https://docs.google.com/spreadsheets/d/12-GISr1efphjtpuJLCfQzI2akNXxaJ1iabsG24ib71c/edit#gid=845083323"

# Read in summary DF and drop empty rows
df = read_gsheet(
    STOCKS_WATCH_URL, 
    header=None
)

In [8]:
# find df header row index using regex pattern
header_idx = df.apply(lambda x: x.str.contains("Neil's Value", case=False)).any(axis='columns').argmax()

# separate disclaimer and df
df_disclaimer = df.iloc[:header_idx-1, 0]
df_watch = df.iloc[header_idx:,]

# set first row as header & reset row idxs
df_watch.columns = df_watch.iloc[0].values
df_watch = df_watch.iloc[1:].reset_index(drop=True)

# Generate buttons for 'Notes' column
df_watch.Notes[df_watch.Notes.notna()] = df_watch.Notes[df_watch.Notes.notna()].apply(comment_button)
df_watch = df_watch.fillna("")

# Color Ratings based on category
rating_colormap = {'Sig Undervalued':'green', 'Mod Undervalued':'blue', 'Fair Value':'grey', 'Value Trap?':'red'}
#df_watch["rating_color"] = df_watch.Rating.map(rating_colormap)


In [30]:
df1 = df_disclaimer.copy()

# add period end of the sentence if not so
df1 = df1.to_frame().rename(columns={0: 'info'})
df1.loc[:,'info'] = [x + '.' if x[-1] != '.' else x for x in df1['info']]

df_disc = df1.copy()

df_disc['color'] = ['warning', 'success', 'warning', 'success', 'success', 'danger']
df_disc['color'] = pd.Categorical(df_disc['color'],
                                  categories=['success', 'warning', 'danger'],
                                  ordered=True)
icon_dict = {'warning': 'exclamation-triangle-fill', 'success': 'check-lg', 'danger':'exclamation-octagon-fill'}
df_disc['icon_id'] = df_disc['color'].map(icon_dict)
df_disc = df_disc.sort_values('color')



,info,color,icon_id
1,Please make your own copy of the stocks sectio...,success,check-lg
3,These stocks were found using the techniques t...,success,check-lg
4,"Neil's Stocks Strategy: 80% into Index funds, ...",success,check-lg
0,These Valuations are for the purpose of Long T...,warning,exclamation-triangle-fill
2,NOTE! Many of the metrics below have to be man...,warning,exclamation-triangle-fill
5,"AT THIS TIME, I AM OUT OF ALL STOCKS. PLEASE D...",danger,exclamation-octagon-fill
